<a href="https://colab.research.google.com/github/Sagramine/ScraptKomenIG/blob/main/Scrapt_IG_Comment_(Final).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Dependencies**

In [ ]:
!pip install instagrapi pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.7/169.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.3/455.3 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.3 MB/s eta 0:00:00
  Created wheel for instagrapi: filename=instagrapi-2.1.3-py3-none-any.whl size=116361 sha256=27484fead9a3eeed92f0235a2c69d9f8e64f55a625127a22ca5f124772a00bf0
  Stored in directory: /root/.cache/pip/wheels/31/c0/63/7ad4ef727751385676d1987f63b99b76c56adbbd043c17aa59
Successfully built instagrapi
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.27.2
    Uninstalling pydantic_core-2.27.2:
      Successfully uninstalled pydantic_core-2.27.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.6

In [ ]:
!pip install --upgrade instagrapi

# **Import Library**

In [ ]:
from instagrapi import Client
import pandas as pd
from datetime import datetime
from getpass import getpass
import time
import random

# **User Login**

In [ ]:
USERNAME = input("Masukkan username Instagram Anda: ")
PASSWORD = getpass("Masukkan password Instagram Anda: ")

# Inisialisasi client
cl = Client()
cl.login(USERNAME, PASSWORD)

print("\nBerhasil login sebagai:", cl.account_info().username)


Masukkan username Instagram Anda: digitaltrackid
Masukkan password Instagram Anda: ··········
Enter code (6 digits) for digitaltrackid (ChallengeChoice.EMAIL): 272597
Code entered "272597" for digitaltrackid (0 attempts by 5 seconds)

Berhasil login sebagai: digitaltrackid


# **Scrapt**

In [ ]:
def get_comments(url, max_comments=100):
    try:
        # Dapatkan media dari URL
        media_pk = cl.media_pk_from_url(url)

        # Ambil komentar
        comments = cl.media_comments(media_pk, amount=max_comments)

        # Format data
        comments_data = []
        try:
            for c in comments:
                comments_data.append({
                    'username': c.user.username,
                    'profile_url': f"https://instagram.com/{c.user.username}",
                    'comment': c.text,
                    'likes': c.like_count,
                    'timestamp': c.created_at_utc.strftime('%Y-%m-%d %H:%M:%S')
                })
                waktu_tunda = random.uniform(1, 4)
                print(f"Menunggu selama {waktu_tunda:.2f} detik...")
                time.sleep(waktu_tunda)
            return comments_data

        except Exception as e:
            print(f"Terjadi kesalahan saat memproses komentar: {e.__class__.__name__} - {str(e)}")
            return []

    except Exception as e:
        print(f"Terjadi kesalahan: {e.__class__.__name__} - {str(e)}")
        return []

# Fungsi untuk mendapatkan shortlink dari URL
def get_shortlink(url):
    # Menghapus '/' di awal dan akhir URL jika ada
    url = url.strip('/')
    # Memecah URL menjadi bagian-bagian
    parts = url.split('/')
    # Shortlink biasanya ada di indeks terakhir
    return parts[-1]

# **Eksekusi**

In [ ]:
from google.colab import files

# Inisialisasi list untuk menyimpan nama file yang akan diunduh
download_filenames = []

# Inisialisasi list untuk menyimpan semua komentar (opsional)
all_comments = []

post_urls = []
shortlinks = []

while True:
    url = input("\nMasukkan URL posting Instagram (atau ketik 'selesai'): ").strip()
    if url.lower() == 'selesai':
        break
    if '/p/' not in url and '/reel/' not in url:
        print("URL tidak valid! Harus mengandung '/p/' atau '/reel/'")
        continue
    post_urls.append(url)
    # Mendapatkan shortlink dan menambahkannya ke daftar
    shortlink = get_shortlink(url)
    shortlinks.append(shortlink)

if not post_urls:
    print("Tidak ada URL yang dimasukkan!")
    exit()

# Input jumlah maksimal komentar
while True:
    try:
        max_comments = int(input("\nMasukkan jumlah maksimal komentar per post: "))
        if max_comments > 0:
            break
        else:
            print("Jumlah maksimal komentar harus lebih dari 0.")
    except ValueError:
        print("Mohon masukkan angka yang valid.")

# Proses setiap URL
for url, shortlink in zip(post_urls, shortlinks):
    print(f"\nMemproses: {url}")
    comments = get_comments(url, max_comments)
    print(f"Jumlah komentar yang dikumpulkan: {len(comments)}")

    if comments:
        df = pd.DataFrame(comments)

        # Konversi tipe data
        df['likes'] = pd.to_numeric(df['likes'])
        df['timestamp'] = pd.to_datetime(df['timestamp'])

        # Sorting data
        df = df.sort_values(by=['likes', 'timestamp'], ascending=[False, False])

        # Simpan ke CSV dengan nama file menggunakan shortlink
        filename = f"ig_comments_{shortlink}.csv"
        df.to_csv(filename, index=False)

        # Tambahkan nama file ke daftar untuk diunduh nanti
        download_filenames.append(filename)

        print(f"\nFile berhasil disimpan: {filename}")
        print("\nPreview Data:")
        print(df.head())

        # Tambahkan komentar ke all_comments (opsional)
        all_comments.extend(comments)
    else:
        print("\nTidak ada komentar yang berhasil dikumpulkan untuk URL ini.")

    # Waktu tunda acak antara 3 hingga 5 detik
    waktu_tunda = random.uniform(3, 5)
    print(f"Menunggu selama {waktu_tunda:.2f} detik sebelum memproses URL berikutnya...")
    time.sleep(waktu_tunda)

# Setelah loop selesai, unduh semua file
for file in download_filenames:
    try:
        files.download(file)
    except ImportError:
        print(f"File disimpan sebagai {file}")

# (Opsional) Buat dan unduh file gabungan semua komentar
if all_comments:
    print(f"\nJumlah total komentar yang dikumpulkan: {len(all_comments)}")
    df_all = pd.DataFrame(all_comments)

    # Konversi tipe data
    df_all['likes'] = pd.to_numeric(df_all['likes'])
    df_all['timestamp'] = pd.to_datetime(df_all['timestamp'])

    # Sorting data
    df_all = df_all.sort_values(by=['likes', 'timestamp'], ascending=[False, False])

    # Simpan ke CSV gabungan
    filename_all = "ig_comments_all.csv"
    df_all.to_csv(filename_all, index=False)

    # Unduh file gabungan
    try:
        files.download(filename_all)
    except ImportError:
        print(f"File gabungan disimpan sebagai {filename_all}")

    print(f"\nFile gabungan berhasil disimpan: {filename_all}")
    print("\nPreview Data Gabungan:")
    print(df_all.head())
else:
    print("\nTidak ada komentar yang berhasil dikumpulkan dari semua URL.")


Masukkan URL posting Instagram (atau ketik 'selesai'): https://www.instagram.com/p/C6DJbZPphVm

Masukkan URL posting Instagram (atau ketik 'selesai'): selesai

Masukkan jumlah maksimal komentar per post: 100

Memproses: https://www.instagram.com/p/C6DJbZPphVm
Menunggu selama 2.15 detik...
Menunggu selama 2.23 detik...
Menunggu selama 1.55 detik...
Menunggu selama 3.13 detik...
Menunggu selama 1.73 detik...
Menunggu selama 3.97 detik...
Menunggu selama 2.39 detik...
Menunggu selama 1.30 detik...
Menunggu selama 2.87 detik...
Menunggu selama 2.96 detik...
Menunggu selama 2.95 detik...
Menunggu selama 3.84 detik...
Menunggu selama 1.27 detik...
Menunggu selama 1.94 detik...
Menunggu selama 3.64 detik...
Menunggu selama 2.78 detik...
Menunggu selama 1.63 detik...
Menunggu selama 2.29 detik...
Menunggu selama 2.55 detik...
Menunggu selama 1.42 detik...
Menunggu selama 3.89 detik...
Menunggu selama 1.45 detik...
Menunggu selama 1.62 detik...
Menunggu selama 3.74 detik...
Menunggu selama 3.7

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Jumlah total komentar yang dikumpulkan: 100


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


File gabungan berhasil disimpan: ig_comments_all.csv

Preview Data Gabungan:
            username                             profile_url  \
83      ahmad_zukhri      https://instagram.com/ahmad_zukhri   
71        sidiq.pram        https://instagram.com/sidiq.pram   
0   firmannugraha481  https://instagram.com/firmannugraha481   
1     fajar_agusta94    https://instagram.com/fajar_agusta94   
2     fajar_agusta94    https://instagram.com/fajar_agusta94   

                                              comment  likes  \
83  Semoga bisa sampai tuntas diberantas judi onli...      2   
71  @danip_122 ngapain d tangkap? Kenyataan nya gi...      1   
0               Tolong berantas judi online semua nya      0   
1                                            @prabowo      0   
2                                  @gibran_rakabuming      0   

             timestamp  
83 2024-04-22 14:10:48  
71 2024-04-22 21:38:00  
0  2024-05-22 08:47:02  
1  2024-05-15 03:47:59  
2  2024-05-15 03:47:55  
